# Unified LLM Generation Pipeline

This notebook provides an interface to the unified generation pipeline.
All logic lives in `generation.py` - this notebook is for interactive testing and batch runs.

The pipeline handles both Paleolithic and Holocene people automatically based on the sampled birth year.

In [1]:
import dill
from tqdm import tqdm

from generation import (
    generate_person,
    generate_batch,
    generate_batch_parallel,
    # Individual steps if needed
    generate_geography,
    generate_demographics,
    generate_structured_incidents,
    generate_historical_context,
    generate_name,
    generate_narrative_plan,
    generate_narrative,
    run_pipeline,
    reset_to_stage
)
from llm_utils import GenerationContext, extract_json

from person import sample_year, sample_person, Person

import copy

# Batch Generation

In [4]:
'''test_people = generate_batch_parallel(n=150, model="gpt-5.2", workers=50)

Sampling 150 people...
Generating 150 people with 50 parallel workers...


100%|█████████████████████████████████████████| 150/150 [13:15<00:00,  5.30s/it]

Done: 150 generated


In [7]:
'''with open('batch2_0100_0249.pkl', 'wb') as f:
    dill.dump(test_people,f)
%run export.py batch2_0100_0249.pkl --start-index 100

Loading people from batch2_0100_0249.pkl...
Found 150 people
Removing existing markdown files from ../_lives_pending...
  Removed 7 files
  Exported 10 people...
  Exported 20 people...
  Exported 30 people...
  Exported 40 people...
  Exported 50 people...
  Exported 60 people...
  Exported 70 people...
  Exported 80 people...
  Exported 90 people...
  Exported 100 people...
  Exported 110 people...
  Exported 120 people...
  Exported 130 people...
  Exported 140 people...
  Exported 150 people...

Successfully exported 150 people to ../_lives_pending
  Index range: 0100 - 0249


# Checking existing generation

In [9]:
test_people[68].messages[:10]

[{'role': 'user',
  'content': 'This is part of a "random lives" project - simulating randomly selecting a person from human history.\n\nFor this historical person, you will be asked demographic questions. For each question, provide a probability distribution\nrepresenting how common each option was among people matching this person\'s known characteristics\n(birth time, location, age, sex, personality, lifestyle, etc.). Each question should be answered conditional on all previously\ngenerated information.\n\nYOUR GOAL: Estimate the TRUE HISTORICAL FREQUENCIES, not what seems interesting or diverse.\n- If 90% of people in this demographic had characteristic X, assign it 90% probability\n- Focus on ordinary people, not exceptional individuals or elites\n- Boring and repetitive answers are often historically correct\n- Some personality extremes can represent substantial limitations and strongly influence a person\'s life trajectory\n\nTECHNICAL REQUIREMENTS:\n- Probabilities must sum to 